In [1]:
import pandas as pd
from pathlib import Path

project_path=Path.cwd().joinpath("project1/Heart_disease_prediction")


In [2]:
%cd project1/Heart_disease_prediction

/home/busybutlazy/project1/Heart_disease_prediction


/home/busybutlazy/miniconda3/envs/heart_diease/lib/python3.11/site-packages/IPython/core/magics/osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [3]:
heart_data = pd.read_csv(project_path.joinpath("dataset/cleaned_merged_heart_dataset.csv"))

In [4]:
heart_data.head()

,age,sex,cp,trestbps,chol,fbs,restecg,thalachh,exang,oldpeak,slope,ca,thal,target
0,63,1,3,145,233,1,0,150,0,2.3,0,0,1,1
1,37,1,2,130,250,0,1,187,0,3.5,0,0,2,1
2,41,0,1,130,204,0,0,172,0,1.4,2,0,2,1
3,56,1,1,120,236,0,1,178,0,0.8,2,0,2,1
4,57,0,0,120,354,0,1,163,1,0.6,2,0,2,1


In [5]:
heart_data.shape

(1888, 14)

In [6]:
column_names = heart_data.columns
print(column_names)

Index(['age', 'sex', 'cp', 'trestbps', 'chol', 'fbs', 'restecg', 'thalachh',
       'exang', 'oldpeak', 'slope', 'ca', 'thal', 'target'],
      dtype='object')


In [7]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from torch.utils.data import DataLoader , TensorDataset

In [8]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [9]:
X,y=heart_data[column_names[:-1]].values,heart_data["target"].values

In [10]:
X[:5]

array([[ 63. ,   1. ,   3. , 145. , 233. ,   1. ,   0. , 150. ,   0. ,
          2.3,   0. ,   0. ,   1. ],
       [ 37. ,   1. ,   2. , 130. , 250. ,   0. ,   1. , 187. ,   0. ,
          3.5,   0. ,   0. ,   2. ],
       [ 41. ,   0. ,   1. , 130. , 204. ,   0. ,   0. , 172. ,   0. ,
          1.4,   2. ,   0. ,   2. ],
       [ 56. ,   1. ,   1. , 120. , 236. ,   0. ,   1. , 178. ,   0. ,
          0.8,   2. ,   0. ,   2. ],
       [ 57. ,   0. ,   0. , 120. , 354. ,   0. ,   1. , 163. ,   1. ,
          0.6,   2. ,   0. ,   2. ]])

In [11]:
y[:5]

array([1, 1, 1, 1, 1])

In [12]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [13]:
scalar=StandardScaler()
X_train=scalar.fit_transform(X_train)
X_test=scalar.transform(X_test)

In [14]:
X_train_tensor=torch.tensor(X_train,dtype=torch.float32)
X_test_tensor=torch.tensor(X_test,dtype=torch.float32)
y_train_tensor=torch.tensor(y_train,dtype=torch.float32).view(-1, 1)
y_test_tensor=torch.tensor(y_test,dtype=torch.float32).view(-1, 1)

In [15]:
X_train_tensor=X_train_tensor.to(device)
X_test_tensor=X_test_tensor.to(device)
y_train_tensor=y_train_tensor.to(device)
y_test_tensor=y_test_tensor.to(device)
print("to deviece")

to deviece


In [16]:
# batch_size=16
# train_dataset=TensorDataset(X_train_tensor,y_train_tensor)
# train_loader = DataLoader(train_dataset,batch_size=batch_size,shuffle=False,pin_memory=True)

In [17]:
class MLPModel(nn.Module):
    def __init__(self,input_size,hidden_size,output_size):
        super(MLPModel,self).__init__()
        self.fc1=nn.Linear(input_size,hidden_size)
        self.relu=nn.ReLU()
        self.fc2=nn.Linear(hidden_size,hidden_size)
        self.fc3=nn.Linear(hidden_size,hidden_size)
        self.fc4=nn.Linear(hidden_size,output_size)
    
    def forward(self,x):
        x=self.fc1(x)
        x=self.relu(x)
        x=self.fc2(x)
        x=self.relu(x)
        x=self.fc3(x)
        x=self.relu(x)
        x=self.fc4(x)
        return x
    


In [18]:
input_size=X_train.shape[1]
hidden_size=20
output_size=1

In [19]:
model = MLPModel(input_size=input_size,hidden_size=hidden_size,output_size=output_size)
model=model.to(device)
model.eval()

MLPModel(
  (fc1): Linear(in_features=13, out_features=20, bias=True)
  (relu): ReLU()
  (fc2): Linear(in_features=20, out_features=20, bias=True)
  (fc3): Linear(in_features=20, out_features=20, bias=True)
  (fc4): Linear(in_features=20, out_features=1, bias=True)
)

In [20]:
criterion = nn.BCEWithLogitsLoss()
optimizer=optim.SGD(model.parameters(),lr=0.001)


In [21]:
# num_epochs=100000
# for epoch in range(num_epochs):
#     epoch_loss = 0.0
#     for batch_idx,(inputs,targets) in enumerate(train_loader):
#         # inputs , targets = inputs.to(device),targets.to(device)
#         outputs = model(inputs)
#         loss=criterion(outputs,targets)    
#         optimizer.zero_grad()
#         loss.backward()
#         optimizer.step()
#         epoch_loss += loss.item()

#     if (epoch+1)%1000==0:
#         avg_loss = epoch_loss / len(train_loader)
#         print(f"Epoch[{epoch+1}/{num_epochs},Loss:{loss.item():.4f}]")

In [28]:
num_epochs=100000
for epoch in range(num_epochs):
    outputs = model(X_train_tensor)
    loss = criterion (outputs,y_train_tensor)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    if (epoch+1)%5000==0:
        print(f"Epoch[{epoch+1}/{num_epochs},Loss:{loss.item():.4f}]")

Epoch[5000/100000,Loss:0.0023]
Epoch[10000/100000,Loss:0.0022]
Epoch[15000/100000,Loss:0.0021]
Epoch[20000/100000,Loss:0.0020]
Epoch[25000/100000,Loss:0.0020]
Epoch[30000/100000,Loss:0.0019]
Epoch[35000/100000,Loss:0.0018]
Epoch[40000/100000,Loss:0.0017]
Epoch[45000/100000,Loss:0.0017]
Epoch[50000/100000,Loss:0.0016]
Epoch[55000/100000,Loss:0.0015]
Epoch[60000/100000,Loss:0.0015]
Epoch[65000/100000,Loss:0.0014]
Epoch[70000/100000,Loss:0.0014]
Epoch[75000/100000,Loss:0.0013]
Epoch[80000/100000,Loss:0.0013]
Epoch[85000/100000,Loss:0.0013]
Epoch[90000/100000,Loss:0.0012]
Epoch[95000/100000,Loss:0.0012]
Epoch[100000/100000,Loss:0.0012]


In [29]:
with torch.no_grad():
    model.eval()
    y_pred = model(X_test_tensor)
    y_pred_label = (torch.sigmoid(y_pred)>0.5).float()
    accuracy = (y_pred_label == y_test_tensor).float().mean()
    print(f"Test accuracy : {accuracy}")

Test accuracy : 0.9497354030609131


In [30]:
torch.save(model.state_dict(), 'pretrained_model_ver2.pth')